In [6]:
# コード更新用ファイル　
# 下記コードを実行するとextract_figure.pyファイルが更新される
!jupyter nbconvert --to python extract_figure.ipynb

In [2]:
import os
import fitz
import re
import pandas as pd

In [3]:
def extract_figure(pdf_folder, pattern, csv_filename):
    """
    フォルダ内の複数のPDFファイルから図番号とその座標を抽出し、データフレームに格納する関数。

    Args:
        pdf_folder (str): PDFファイルが格納されているフォルダのパス。
        pattern (str): 図番号の正規表現パターン。
        csv_filename (str): 出力csvファイルのパス

    """
    # データフレームのためのリストを初期化
    data = []

    # フォルダ内のPDFファイルのリストを取得
    pdf_files = [os.path.join(pdf_folder, filename) for filename in os.listdir(pdf_folder) if filename.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_document = fitz.open(pdf_file)

        for page_number in range(len(pdf_document)):
            page = pdf_document[page_number]
            pdf_text = page.get_text()
            blocks = page.get_text("blocks")

            for block in blocks:
                text = block[4]  # テキスト部分を取得
                match = re.search(pattern, text)
                if match:
                        x0, y0, x1, y1 = block[:4]  # ブロックの座標を取得

                        # マッチした部分の開始・終了位置を取得
                        start_pos = match.start()
                        end_pos = match.end()

                        # 開始・終了位置の座標を調整
                        x0 = x0 + start_pos  # これは簡易的な方法であり、実際には適切な位置を計算する必要がある
                        x1 = x0 + end_pos    # 同上

                        print(f"Matched text: {match.group()} at coordinates: ({x0}, {y0}), ({x1}, {y1})")

                        data.append({
                            'PDFファイル': os.path.basename(pdf_file),
                            'ページ番号': page_number + 1,
                            '図番号': match.group(),
                            'x0': x0,
                            'y0': y0,
                            'x1': x1,
                            'y1': y1
                        })

        pdf_document.close()

    # データフレームに情報を格納
    df = pd.DataFrame(data)
    
    # データフレームをCSVファイルとして保存
    df.to_csv(csv_filename, index=False)

    return df


In [4]:
# 使用例:
if __name__ == "__main__":
    pdf_folder = '../sample-pdf'  # PDFファイルが格納されているフォルダのパス
    pattern = r"【図 [0-9]+\-[0-9]+】"  # 図番号の正規表現パターン
    figure_csv_filename = 'figure_info.csv'
    result_df = extract_figure(pdf_folder, pattern, figure_csv_filename)


Matched text: 【図 1-1】 at coordinates: (84.95994567871094, 339.4860534667969), (91.95994567871094, 351.6526794433594)
Matched text: 【図 1-2】 at coordinates: (84.95994567871094, 282.8460693359375), (91.95994567871094, 295.0126953125)
Matched text: 【図 1-3】 at coordinates: (84.95994567871094, 169.3260498046875), (91.95994567871094, 181.49269104003906)
Matched text: 【図 1-4】 at coordinates: (84.95994567871094, 339.4860534667969), (91.95994567871094, 351.6526794433594)
Matched text: 【図 1-5】 at coordinates: (84.95994567871094, 569.4060668945312), (91.95994567871094, 581.57275390625)
Matched text: 【図 1-6】 at coordinates: (84.95994567871094, 368.52606201171875), (91.95994567871094, 380.69268798828125)
Matched text: 【図 1-7】 at coordinates: (84.95994567871094, 188.2860565185547), (91.95994567871094, 200.45269775390625)
Matched text: 【図 1-8】 at coordinates: (84.95994567871094, 387.2460632324219), (91.95994567871094, 399.4126892089844)
Matched text: 【図 2-1】 at coordinates: (84.95994567871094, 259.930